# **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4835292633514952>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import *

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
     70     self._should_hint = True
---> 71 module = self._original_builtins_import(name, *args, **kwargs)
     72 if (fname := fname or get_allowed_file_name_or_none(module)) is not None:
     73     mtime = mtime or os.stat(fname).st_mtime

### Streaming Table

In [0]:
# Expectations
my_rules = {
    "rule1": 'product_id IS NOT NULL',
    'rule2': 'product_name IS NOT NULL'
}

In [0]:
@dlt.table()

@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
    df = spark.readStream.table('databricks_catalog.silver.products_silver')
    return df

### Streaming View

In [0]:
@dlt.view()
def Dimproducts_view():
    df = spark.readStream.table('Live.DimProducts_stage')
    return df 

### DimProducts

In [0]:
dlt.create_streaming_table('DimProducts')

In [0]:
dlt.apply_changes(
    target = 'DimProducts',
    source = 'Live.DimProducts_view',
    keys = ['product_id'],
    sequence_by = 'product_id',
    stored_as_scd_type = 2
)